In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from crewai import LLM
llm = LLM(
  model='gpt-4o-mini',
  temperature=0.1
)


In [3]:
import os
import requests
from dotenv import load_dotenv
from crewai.tools import BaseTool


class ScrapeNewsTool(BaseTool):
    name: str = "Scrape for news"
    description: str = "Uses GNews API to fetch latest local news"

    def _run(self, query: str) -> str:
        try:
            api_key = os.getenv("GNEWS_API_KEY")
            if not api_key:
                return "GNEWS_API_KEY not found in environment."

            url = f"https://gnews.io/api/v4/search?q={query}&lang=en&country=in&max=5&apikey={api_key}"
            response = requests.get(url)

            if response.status_code != 200:
                return f"Failed to fetch news. Status code: {response.status_code}"

            data = response.json()
            articles = data.get("articles", [])

            if not articles:
                return "No articles found."

            news_list = []
            for i, article in enumerate(articles, 1):
                title = article["title"]
                link = article["url"]
                news_list.append(f"{i}. {title}\n{link}")

            return "\n\n".join(news_list)

        except Exception as e:
            return f"An error occurred while fetching news: {e}"


In [4]:
from crewai.tools import BaseTool
from crewai import LLM

llm = LLM(model="gpt-4o-mini", temperature=0.3)

class SummarizerTool(BaseTool):
    name: str = "Summarizer"
    description: str = "Summarizes long news articles into concise summaries"

    def _run(self, article_text: str) -> str:
        prompt = f"""You are a professional news summarizer.
Summarize the following article in 2-3 bullet points:
        
{article_text}"""
        try:
            return llm.call(prompt)
        except Exception as e:
            return f"Error summarizing article: {e}"


In [5]:
class ClassifierTool(BaseTool):
    name: str = "News Classifier"
    description: str = "Classifies news into categories like Politics, Crime, Sports, Entertainment, Business, Technology, Health, Education, Other."

    def _run(self, summary: str) -> str:
        prompt = f"""Classify the following news summary into one of the categories:
[Politics, Crime, Sports, Entertainment, Business, Technology, Health, Education, Other]

News Summary:
{summary}

Respond with only the category."""
        try:
            return llm.call(prompt).strip()
        except Exception as e:
            return f"Error classifying news: {e}"


In [6]:
from crewai import Agent

scraper_agent = Agent(
    role="News Scraper",
    goal="Find the latest local news headlines",
    backstory="You're responsible for fetching news articles from the web based on a query.",
    tools=[ScrapeNewsTool()],
    verbose=True
)

summarizer_agent = Agent(
    role="News Summarizer",
    goal="Summarize long-form news articles",
    backstory="You're a journalist trained to condense news into readable points.",
    tools=[SummarizerTool()],
    verbose=True
)

classifier_agent = Agent(
    role="News Classifier",
    goal="Categorize summarized news articles",
    backstory="You're a news analyst responsible for tagging articles for newsletters.",
    tools=[ClassifierTool()],
    verbose=True
)


In [7]:
from crewai import Task, Crew

# 1. Scraping Task → assigned to scraper_agent
scrape_task = Task(
    description="Scrape the latest local news about India using the tool.",
    expected_output="A list of top 5 full news article headlines and their content or summaries.",
    agent=scraper_agent
)

# 2. Summarizing Task → assigned to summarizer_agent
summarize_task = Task(
    description="Summarize each of the news articles scraped by the scraper agent.",
    expected_output="2-3 bullet point summaries for each article.",
    agent=summarizer_agent
)

# 3. Classification Task → assigned to classifier_agent
classify_task = Task(
    description="Classify each news summary into a category: Politics, Crime, Sports, Entertainment, Business, Technology, Health, Education, or Other.",
    expected_output="Each news summary with an associated category.",
    agent=classifier_agent
)

# Now create the crew with those tasks
news_crew = Crew(
    agents=[scraper_agent, summarizer_agent, classifier_agent],
    tasks=[scrape_task, summarize_task, classify_task],
    verbose=True
)

# Run the crew
result = news_crew.kickoff()
print(result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 15de6990-feac-4a49-9b1a-88464160612b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: News Scraper                                                                                            │
│                                                                                                                 │
│  Task: Scrape the latest local news about India using the tool.                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\Satvik\Documents\VS Code projects\Web\CrewAI\summerschool\.venv\Lib\site-packages\rich\live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: News Scraper                                                                                            │
│                                                                                                                 │
│  Thought: I need to fetch the latest local news about India using the available tool.                           │
│                                                                                                                 │
│  Using Tool: Scrape for news                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"local news India\"}"                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Technology                                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: News Classifier                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Operation Mahadev: Decoding the Pakistan Link in Pahalgam Attack** - Politics                             │
│  2. **Iranian Embassy Urges Indian Media to Uphold Journalistic Integrity** - Politics                          │
│  3. **Breaking Health News: EMA Approves Nasal Spray, India Boosts Diabetes Drug Production** - Health          │
│  4. **PoK protests are good news for India. Shows local issues are more important than Kashmir** - Politics     │
│  5. **After WhatsApp, Meta now testing its AI-powered chatbot in Instagram: All you need to know** -            │
│  Technology                                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8842b33f-d864-4bce-9304-9858feeab6f3                                                                     │
│  Agent: News Classifier                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 15de6990-feac-4a49-9b1a-88464160612b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. **Operation Mahadev: Decoding the Pakistan Link in Pahalgam Attack** - Politics               │
│  2. **Iranian Embassy Urges Indian Media to Uphold Journalistic Integrity** - Politics                          │
│  3. **Breaking Health News: EMA Approves Nasal Spray, India Boosts Diabetes Drug Production** - Health          │
│  4. **PoK protests are good news for India. Shows local issues are more important than Kashmir** - Politics     │
│  5. **After WhatsApp, Meta now testing its AI-powered chatbot in Instagram: All you need to know** -            │
│  Technology                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

1. **Operation Mahadev: Decoding the Pakistan Link in Pahalgam Attack** - Politics  
2. **Iranian Embassy Urges Indian Media to Uphold Journalistic Integrity** - Politics  
3. **Breaking Health News: EMA Approves Nasal Spray, India Boosts Diabetes Drug Production** - Health  
4. **PoK protests are good news for India. Shows local issues are more important than Kashmir** - Politics  
5. **After WhatsApp, Meta now testing its AI-powered chatbot in Instagram: All you need to know** - Technology
